In [ ]:
import pandas
import matplotlib.pyplot as plt

In [ ]:
df = pandas.read_excel('output/results.xlsx', sheet_name="data", index_col=0)

In [ ]:
# Set up line styles
black = "#000000"
colors = [
    "#0070a3",
    "#5273ba",
    "#916fc2",
    "#ca67b7",
    "#f6619c",
    "#ff6974",
    "#ff8347",
    "#ffa600",
]

style_map = {
    'Virtual Best': dict(color=black, linestyle="solid"),
    'BARON-BM': dict(color=colors[0], linestyle="dashed"),
    'BARON-HR': dict(color=colors[0], linestyle="dotted"),
    'SCIP-BM': dict(color=colors[1], linestyle="dashed"),
    'SCIP-HR': dict(color=colors[1], linestyle="dotted"),
    'DICOPT-BM': dict(color=colors[2], linestyle="dashed"),
    'DICOPT-HR': dict(color=colors[2], linestyle="dotted"),
    'GDPopt-GLOA': dict(color=colors[3], linestyle="solid"),
    'GDPopt-GLOA-DVB': dict(color=colors[3], linestyle="dashed"),
    'GDPopt-LBB': dict(color=colors[4], linestyle="dashed"),
    'GDPopt-LOA': dict(color=colors[5], linestyle="dashed"),
    'Mindtpy-OA-BM': dict(color=colors[6], linestyle="dashed"),
    'Mindtpy-OA-HR': dict(color=colors[6], linestyle="dotted"),
}

In [ ]:
# Determine best time to solve for each model
best_soln_times = {model: df.loc[df.model == model].time_to_soln.min() for model in df.model.unique()}

# Eliminate duplicates (take the best time for each model/solver pair)
df_time_to_soln = df.sort_values(by='time_to_soln')
df_time_to_soln = df_time_to_soln.drop_duplicates(subset=['model', 'solver'], keep='first')
df_time_to_soln = df_time_to_soln[['model', 'solver', 'time_to_soln']]

# Generate instances solved vs. time plot
df_time_to_soln = pandas.concat([df_time_to_soln, pandas.DataFrame.from_records(
    list(dict(model=model, solver='Virtual Best', time_to_soln=best_soln_times[model]) for model in best_soln_times),
    index=range(len(best_soln_times))
)], sort=False)
df_time_to_soln = df_time_to_soln[['solver', 'time_to_soln']].sort_values(by='time_to_soln')
for solver in df_time_to_soln.solver.unique():
    # Create columns corresponding to each solution strategy
    df_time_to_soln[solver] = df_time_to_soln.apply(lambda row: 1 if row['solver'] == solver else 0, axis=1)
    df_time_to_soln[solver] = df_time_to_soln[solver].cumsum()
df_time_to_soln = df_time_to_soln.fillna(value=float('inf'))
df_time_to_soln = df_time_to_soln.loc[df_time_to_soln['time_to_soln'] <= 1e6]
df_time_to_soln = df_time_to_soln[['time_to_soln', ] + list(df_time_to_soln.solver.unique())]
# Remove duplicate times. Data columns are cumulative sums, so keep last duplicate row.
df_time_to_soln = df_time_to_soln.drop_duplicates(subset='time_to_soln', keep='last').set_index('time_to_soln')
df_time_to_soln = df_time_to_soln.sort_values(df_time_to_soln.last_valid_index(), axis=1, ascending=False)
plt.figure(dpi=300)
for solver_col in df_time_to_soln.columns:
    plt.plot(df_time_to_soln[solver_col], **style_map[solver_col])
plt.ylim(bottom=0, top=len(best_soln_times))
plt.suptitle('Time to best known solution')
plt.xlabel('Time (seconds)')
plt.ylabel('Instances solved')
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False)
plt.savefig('output/time-to-soln.png', bbox_inches='tight')
with pandas.option_context(
        'display.max_rows', None, 'display.max_columns', None, 'expand_frame_repr', False
), open('output/time-to-soln.log', 'w') as resultsfile:
    print(df_time_to_soln, file=resultsfile)

In [ ]:
# Determine the best time to optimality for each model
best_opt_times = {model: df.loc[df.model == model].time_to_opt.min() for model in df.model.unique()}

# Eliminate duplicates (take the best time for each model/solver pair)
df_time_to_opt = df.sort_values(by='time_to_opt')
df_time_to_opt = df_time_to_opt.drop_duplicates(subset=['model', 'solver'], keep='first')
df_time_to_opt = df_time_to_opt[['model', 'solver', 'time_to_opt']]

# Generate instances solved vs. time plot
df_time_to_opt = pandas.concat([df_time_to_opt, pandas.DataFrame.from_records(
    list(dict(model=model, solver='Virtual Best', time_to_opt=best_opt_times[model]) for model in best_opt_times),
    index=range(len(best_opt_times))
)], sort=False)
df_time_to_opt = df_time_to_opt[['solver', 'time_to_opt']].sort_values(by='time_to_opt')
for solver in df_time_to_opt.solver.unique():
    # Create columns corresponding to each solution strategy
    df_time_to_opt[solver] = df_time_to_opt.apply(lambda row: 1 if row['solver'] == solver else 0, axis=1)
    df_time_to_opt[solver] = df_time_to_opt[solver].cumsum()
df_time_to_opt = df_time_to_opt.fillna(value=float('inf'))
df_time_to_opt = df_time_to_opt.loc[df_time_to_opt['time_to_opt'] <= 1e6]
df_time_to_opt = df_time_to_opt[['time_to_opt', ] + list(df_time_to_opt.solver.unique())]
# Remove duplicate times. Data columns are cumulative sums, so keep last duplicate row.
df_time_to_opt = df_time_to_opt.drop_duplicates(subset='time_to_opt', keep='last').set_index('time_to_opt')
df_time_to_opt = df_time_to_opt.sort_values(df_time_to_opt.last_valid_index(), axis=1, ascending=False)
plt.figure(dpi=300)
for solver_col in df_time_to_opt.columns:
    plt.plot(df_time_to_opt[solver_col], **style_map[solver_col])
plt.ylim(bottom=0, top=len(best_soln_times))
plt.suptitle('Time to global optimality')
plt.xlabel('Time (seconds)')
plt.ylabel('Instances solved')
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False)
plt.savefig('output/time-to-opt.png', bbox_inches='tight')
with pandas.option_context(
        'display.max_rows', None, 'display.max_columns', None, 'expand_frame_repr', False
), open('output/time-to-opt.log', 'w') as resultsfile:
    print(df_time_to_opt, file=resultsfile)

In [ ]:
# Determine the best time to an ok solution (10% gap) for each model
best_ok_times = {model: df.loc[df.model == model].time_to_ok_soln.min() for model in df.model.unique()}

# Eliminate duplicates (take the best time for each model/solver pair)
df_time_to_ok = df.sort_values(by='time_to_ok_soln')
df_time_to_ok = df_time_to_ok.drop_duplicates(subset=['model', 'solver'], keep='first')
df_time_to_ok = df_time_to_ok[['model', 'solver', 'time_to_ok_soln']]

# Generate instances solved vs. time plot
df_time_to_ok = pandas.concat([df_time_to_ok, pandas.DataFrame.from_records(
    list(dict(model=model, solver='Virtual Best', time_to_ok_soln=best_ok_times[model]) for model in best_ok_times),
    index=range(len(best_ok_times))
)], sort=False)
df_time_to_ok = df_time_to_ok[['solver', 'time_to_ok_soln']].sort_values(by='time_to_ok_soln')
for solver in df_time_to_ok.solver.unique():
    # Create columns corresponding to each solution strategy
    df_time_to_ok[solver] = df_time_to_ok.apply(lambda row: 1 if row['solver'] == solver else 0, axis=1)
    df_time_to_ok[solver] = df_time_to_ok[solver].cumsum()
df_time_to_ok = df_time_to_ok.fillna(value=float('inf'))
df_time_to_ok = df_time_to_ok.loc[df_time_to_ok['time_to_ok_soln'] <= 1e6]
df_time_to_ok = df_time_to_ok[['time_to_ok_soln', ] + list(df_time_to_ok.solver.unique())]
# Remove duplicate times. Data columns are cumulative sums, so keep last duplicate row.
df_time_to_ok = df_time_to_ok.drop_duplicates(subset='time_to_ok_soln', keep='last').set_index('time_to_ok_soln')
df_time_to_ok = df_time_to_ok.sort_values(df_time_to_ok.last_valid_index(), axis=1, ascending=False)
plt.figure(dpi=300)
for solver_col in df_time_to_ok.columns:
    plt.plot(df_time_to_ok[solver_col], **style_map[solver_col])
plt.ylim(bottom=0, top=len(best_soln_times))
plt.suptitle('Time to acceptable solution')
plt.xlabel('Time (seconds)')
plt.ylabel('Instances solved')
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False)
plt.savefig('output/time-to-ok-soln.png', bbox_inches='tight')
with pandas.option_context(
        'display.max_rows', None, 'display.max_columns', None, 'expand_frame_repr', False
), open('output/time-to-ok-soln.log', 'w') as resultsfile:
    print(df_time_to_ok, file=resultsfile)

In [ ]:
number_solved = pandas.concat([df_time_to_opt.tail(1), df_time_to_soln.tail(1), df_time_to_ok.tail(1)], axis=0, ignore_index=True, sort=True).fillna(0).astype('int32')
number_solved.set_index(pandas.Index(["1% gap optimal", "1% gap best known", "10% gap best known",]), inplace=True)
number_solved.sort_values(by=["1% gap optimal", "1% gap best known", "10% gap best known"], axis=1, inplace=True, ascending=False)
number_solved = number_solved.transpose()
percent_solved = number_solved.apply(lambda x: round(x / len(best_soln_times) * 100)).astype('int32')

In [ ]:
(number_solved
 .style
 .bar(color='lightgreen', vmin=0, vmax=len(best_soln_times))
 .set_caption("Number of instances solved out of %s" % len(best_soln_times)))

In [ ]:
(percent_solved
 .style
 .bar(color='lightgreen', vmin=0, vmax=100)
 .set_caption("Percent of instances solved out of %s" % len(best_soln_times)))

In [ ]:
# Output the error messages found in the results
fail_messages = (
    df[df.err_msg.notnull()][['model', 'solver', 'err_msg']]
    .sort_values(by=['model', 'solver'])
    .drop_duplicates(subset=['model', 'solver'], keep='first'))
fail_messages.set_index(['model', 'solver'], inplace=True)
fail_messages.style.set_caption("Error messages by model and solver")

In [ ]:
def generate_solution_rank_table(sort_column='time_to_ok_soln'):
    best_solver_df = df.sort_values(by=sort_column).drop_duplicates(subset=['model', 'solver'], keep='first')
    # df.groupby('model').apply(lambda x: print(x[['model', 'solver', 'time_to_ok_soln']]))
    best_solver_df['rank'] = best_solver_df.groupby('model').cumcount()
    best_solver_df['rank'] += 1
    best_solver_df['solver_time'] = best_solver_df.apply(
        lambda row: (row['solver'] + (" %.2f" % row[sort_column]) if pandas.notna(row[sort_column]) else ""), axis=1)
    return (best_solver_df[['model', 'solver', sort_column, 'rank', 'solver_time']]
    #  .groupby('model').head(3)  # Only display top N
     .sort_values(by=['model', sort_column])
     .pivot(index='model', columns='rank', values='solver_time')
     .fillna("")
     .style.set_caption("Best %s" % sort_column.replace("_", " "))
    )

generate_solution_rank_table(sort_column='time_to_ok_soln')
# generate_solution_rank_table(sort_column='time_to_soln')
# generate_solution_rank_table(sort_column='time_to_opt')

In [ ]:
df[df['model'] == 'stickies']